In [2]:
using LinearAlgebra
using SparseArrays
include("arnoldi.jl")
include("bArnoldi.jl")

bArnoldi (generic function with 1 method)

In [3]:
n = 6
M = 3
L = 2
#^^^^^^^^^^^^^^^^^^^^^^^#
#  A*X_σ - X_σ * D = B  #
#_______________________#
# LHS
#   A
λ = [i+10 for i in 1:n]
a = randn(n, n)
A = UpperTriangular(a) - diagm(diag(a)) + diagm(λ)
#   Shift
σ = rand(1:9, L)
D = diagm(σ) 
# RHS
B = randn(n, L);

In [4]:
# Initialize
X_σ     = zeros(n, L) 
X_exact = zeros(n, L)
Res     = zeros(M, L);

In [6]:
bArnoldi(A, B, M, L);